# Install Neccessary Modules

In [11]:
%pip install -qU langchain-groq
%pip install --upgrade --quiet  \
    langchain-pinecone \
    langchain-openai \
    langchain \
    langchain-community \
    pinecone-notebooks
%pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


# Load Neccessary Modules

In [29]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from pinecone import Pinecone, ServerlessSpec
import time
from langchain_community.embeddings import SentenceTransformerEmbeddings
from tqdm.autonotebook import tqdm
from langchain_pinecone import PineconeVectorStore

# Load the API keys

In [13]:
GROQ_API_KEY="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"
pinecone_api_key="11af13a8-46b7-467d-b143-94718f9c92c8"

# Create index in pinecone if it doesn't exist:
This is an user defined function used to create index in pinecone if it it not available

In [26]:
def create_index_in_pinecone(index_name='chat-history'):  # change if desired
    try:
# index_name = "assignment"  # change if desired
        pc = Pinecone(api_key="11af13a8-46b7-467d-b143-94718f9c92c8")
        existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
        
        if index_name not in existing_indexes:
            pc.create_index(
                name=index_name,
                dimension=384,
                metric="cosine",
                spec=ServerlessSpec(cloud="aws", region="us-east-1"),
            )
            while not pc.describe_index(index_name).status["ready"]:
                time.sleep(1)
        
        index = pc.Index(index_name)
        return True
    except:
        return False

# Add the converations to Index
Add the conversation into the index by add_text

In [15]:
 def add_embeddings_to_pinecone(history, index_name='chat-history'):
     model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
     vectorstore = PineconeVectorStore(pinecone_api_key=pinecone_api_key,index_name=index_name, embedding=model)
     for i in history:
         vectorstore.add_texts([i])
     return vectorstore

Dynamic relavent memory selection, this enables to select messages based on the query.

In [40]:
 def retrieve_relevant_history(query, vectorstore, index_name='chat-history'):
     relavent_messages=vectorstore.similarity_search(query)
     messages_string=''
     for i in range(len(relavent_messages)):
         messages_string+=(relavent_messages[i].page_content+' ')
     return messages_string
     

In [98]:
def prepare_prompt(test_prompt, history, index_name='chat-history'):
    chat = ChatGroq(
    groq_api_key=GROQ_API_KEY,
    temperature=0,
    model="mixtral-8x7b-32768",
)


    system = "You are a Prompt generator, you need to generate me a prompt{text} in the lowest tokens possible. Make sure to generate me this prompt for Question and answering, also mention to generate response in short in the prompt you generate"
    human = "{text}"
    prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])
    
    chain = prompt | chat
    test_prompt+=history
    final_prompt=chain.invoke({"text": test_prompt})
    return final_prompt.content

# Prompt Testing
Testing the prompt and evaluate the result.

In [80]:
def test_final_prompt(final_prompt,query):
    chat = ChatGroq(
    groq_api_key=GROQ_API_KEY,
    temperature=0,
    model="mixtral-8x7b-32768",
)


    system = final_prompt
    human = "{text}"
    prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])
    
    chain = prompt | chat
    response=chain.invoke({"text": query})
    return response.content
    

# Load the history conversation

In [17]:
# Define the conversation history
history = [
 "1: User: Hi there! How are you doing today? | Bot: Hello! I'm doing great, thank you! How can I assist you today?",
 "2: User: What's the weather like today in New York? | Bot: Today in New York, it's sunny with a slight chance of rain.",
 "3: User: Great! Do you have any good lunch suggestions? | Bot: Sure! How about trying a new salad recipe?",
 "4: User: That sounds healthy. Any specific recipes? | Bot: You could try a quinoa salad with avocado and chicken.",
 "5: User: Sounds delicious! I'll try it. What about dinner? | Bot: For dinner, you could make grilled salmon with vegetables.",
 "6: User: Thanks for the suggestions! Any dessert ideas? | Bot: How about a simple fruit salad or yogurt with honey?",
 "7: User: Perfect! Now, what are some good exercises? | Bot: You can try a mix of cardio and strength training exercises.",
 "8: User: Any specific recommendations for cardio? | Bot: Running, cycling, and swimming are all excellent cardio exercises.",
 "9: User: I'll start with running. Can you recommend any books? | Bot: 'Atomic Habits' by James Clear is a highly recommended book.",
 "10: User: I'll check it out. What hobbies can I take up? | Bot: You could explore painting, hiking, or learning a new instrument.",
 "11: User: Hiking sounds fun! Any specific trails? | Bot: There are great trails in the Rockies and the Appalachian Mountains.",
 "12: User: I'll plan a trip. What about indoor activities? | Bot: Indoor activities like reading, cooking, or playing board games.",
 "13: User: Nice! Any good board games? | Bot: Settlers of Catan and Ticket to Ride are both excellent choices.",
 "14: User: I'll try them out. Any movie recommendations? | Bot: 'Inception' and 'The Matrix' are must-watch movies.",
 "15: User: I love those movies! Any TV shows? | Bot: 'Breaking Bad' and 'Stranger Things' are very popular.",
 "16: User: Great choices! What about podcasts? | Bot: 'How I Built This' and 'The Daily' are very informative.",
 "17: User: Thanks! What are some good travel destinations? | Bot: Paris, Tokyo, and Bali are amazing travel spots.",
 "18: User: I'll add them to my list. Any packing tips? | Bot: Roll your clothes to save space and use packing cubes.",
 "19: User: That's helpful! What about travel insurance? | Bot: Always get travel insurance for safety and peace of mind.",
 "20: User: Thanks for the tips! Any last advice? | Bot: Just enjoy your journey and make the most out of your experiences."
]

In [30]:
status=create_index_in_pinecone(index_name='chat-history')
if status:
    vectorstore=add_embeddings_to_pinecone(history)

In [31]:
vectorstore

In [82]:
query="Get me about fitness"

In [83]:
messages=retrieve_relevant_history(query, vectorstore)

In [99]:
test_prompt="you are my helpful assistant, you need to answer based on the query and history_messages "
prompt=prepare_prompt(test_prompt, messages, index_name='chat-history')


In [100]:
prompt=prompt.split('"')[-2]

In [101]:
response=test_final_prompt(prompt,query)

In [102]:

print("Test Prompt:\n\n")
print(test_prompt)
print("\n\n")
print("Final Prompt:\n\n")
print(prompt)
print("\n\n")
print("Response:\n\n")
print(response)

Test Prompt:


you are my helpful assistant, you need to answer based on the query and history_messages 



Final Prompt:


You are my helpful assistant. Generate a response to the user's query about hobbies based on the previous history messages. Keep the response short.

Previous messages:
7: User: Perfect! Now, what are some good exercises? | Bot: You can try a mix of cardio and strength training exercises.
8: User: Any specific recommendations for cardio? | Bot: Running, cycling, and swimming are all excellent cardio exercises.
9: User: I'll start with running. Can you recommend any books? | Bot: 'Atomic Habits' by James Clear is a highly recommended book.
10: User: I'll check it out.

User: What hobbies can I take up?
Assistant: You could explore painting, hiking, or learning a new instrument.



Response:


Based on our previous conversation, it seems you're interested in self-improvement and physical activities. In addition to running and reading, you might consider taking up ho